In [307]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import *
from sklearn.metrics.pairwise import *

from sklearn.preprocessing import *
from sklearn.model_selection import *
from sklearn.linear_model import *
from sklearn.metrics import *

In [308]:
df = pd.read_excel("IELTS_Success_Stories_Dataset.xlsx")

In [309]:
df.head()

,Candidate,Location,Profession,Study Duration (months),IELTS Score (Overall),Key Strategies,Education Level,Age,Target Country,English Proficiency (Preparation),...,Preferred Learning Resources,Application Status,Job Offer Received,Additional Certifications,Volunteer Experience,Language Fluency,Internship Experience,Relevant Skills,Recommendations,Networking Efforts
0,Aisha,Karachi,Grad Student,3,8.50,"Practice, forums, tutoring",Master's,24,Canada,Advanced,...,"Online courses, study guides",Under Process,No,TESOL,Yes,"Urdu, English",Research Assistant,"Data Analysis, Teaching",Strong,"LinkedIn, Local Events"
1,Ahmed,Lahore,IT Professional,4,7.50,"Balanced schedule, time management, speaking p...",Bachelor's,29,Australia,Intermediate,...,"Online forums, practice tests",Approved,Yes,PMP,No,"Urdu, English",IT Consultant,"Project Management, Programming",Positive,Online Communities
2,Fatima,Islamabad,School Teacher,2,9.00,"Intensive course, online resources, mock inter...",Master's,32,New Zealand,Advanced,...,"Study guides, language exchange",Under Process,No,CELTA,Yes,"Urdu, English",Curriculum Developer,"Lesson Planning, Educational Tech",Positive,Professional Groups
3,Ali,Peshawar,IT Professional,5,8.00,"Focused study, workshops, peer feedback",Bachelor's,27,USA,Advanced,...,"Mock exams, speaking clubs",Rejected,No,AWS Certified Solutions Architect,No,"Urdu, English",Software Developer,"Cloud Computing, Programming",Strong,Online Platforms
4,Saad,Lahore,Eng. Student,6,8.25,"Consistent practice, private tutor, speaking c...",Bachelor's,22,UK,Advanced,...,"Online tutorials, speaking practice",Under Process,No,IELTS Examiner,Yes,"Urdu, English",Research Assistant,"Lab Techniques, Data Analysis",Positive,University Events


In [350]:
def retrieve_no_of_columns_to_split(x):
    pp = []
    ctr = 0
    for i in df[x].unique():
        pp += i.split(",")
        ctr += 1
    qq = []
    for i in pp:
        qq.append(i.strip().lower())
    return len(qq)//ctr

def round_to_nearest_ielts_score(x):
    inte = int(x)
    dec = x - inte
    if inte >= 9:
        res = 9.0
    elif dec < 0.25:
        res = float(inte)
    elif dec >= 0.25 and dec < 0.75:
        res = inte + 0.5
    elif dec >= 0.75:
        res = float(inte + 1)
    return res

In [351]:
columns_to_split = ["Key Strategies","Preferred Learning Resources"]
columns_to_remove = ["Candidate","Location","Target Country","Application Status","Job Offer Received","Additional Certifications","Language Fluency","Relevant Skills","Age"]

In [352]:
cols = df.columns.values.tolist()
columns_new = []
start = 0
df_1 = df.copy()
newest_columns = []
for i in columns_to_split:
    data = df[i].copy()
    d = np.array(list(map(lambda x: [i.strip().lower() for i in x.split(", ")],data)))
    ctr = retrieve_no_of_columns_to_split(i)
    new_col = []
    for j in range(ctr):
        ne = i+" "+str(j)
        df_1[ne] = d[:,j]
        new_col.append(ne)
        newest_columns.append(ne)
    stop = cols.index(i)
    columns_new += cols[start:stop]
    columns_new += new_col
    start = stop + 1
columns_new += cols[start:]
df_2 = df_1[columns_new].copy()
df_2.drop(columns=columns_to_remove,inplace=True)
df_2["IELTS Score (Overall)"] = list(map(lambda x: round_to_nearest_ielts_score(x),df_2["IELTS Score (Overall)"]))
correct_order_col = []
for i in df_2.columns:
    if i != "IELTS Score (Overall)":
        correct_order_col.append(i)
correct_order_col.append("IELTS Score (Overall)")
df_2 = df_2[correct_order_col]

In [353]:
df_2.head()

,Profession,Study Duration (months),Key Strategies 0,Key Strategies 1,Key Strategies 2,Education Level,English Proficiency (Preparation),Practice Hours per Week,Mock Tests Taken,Achieved Desired Score,Preferred Learning Resources 0,Preferred Learning Resources 1,Volunteer Experience,Internship Experience,Recommendations,Networking Efforts,IELTS Score (Overall)
0,Grad Student,3,practice,forums,tutoring,Master's,Advanced,15,5,Yes,online courses,study guides,Yes,Research Assistant,Strong,"LinkedIn, Local Events",8.5
1,IT Professional,4,balanced schedule,time management,speaking practice,Bachelor's,Intermediate,10,3,Yes,online forums,practice tests,No,IT Consultant,Positive,Online Communities,7.5
2,School Teacher,2,intensive course,online resources,mock interviews,Master's,Advanced,20,8,Yes,study guides,language exchange,Yes,Curriculum Developer,Positive,Professional Groups,9.0
3,IT Professional,5,focused study,workshops,peer feedback,Bachelor's,Advanced,12,4,Yes,mock exams,speaking clubs,No,Software Developer,Strong,Online Platforms,8.0
4,Eng. Student,6,consistent practice,private tutor,speaking clubs,Bachelor's,Advanced,18,6,Yes,online tutorials,speaking practice,Yes,Research Assistant,Positive,University Events,8.5


In [354]:
key_strategies_initial = list(set(df_2["Key Strategies 0"].unique().tolist() + df_2["Key Strategies 1"].unique().tolist() + df_2["Key Strategies 2"].unique().tolist()))

preferred_learning_resources_initial = list(set(df_2["Preferred Learning Resources 0"].unique().tolist() + df_2["Preferred Learning Resources 1"].unique().tolist()))

initial_key_preferred = []
initial_key_preferred.append(key_strategies_initial)
initial_key_preferred.append(preferred_learning_resources_initial)

In [355]:
key_strategies_initial

['online resources',
 'study groups',
 'forums',
 'focused study',
 'tutoring',
 'speaking with native speakers',
 'balanced schedule',
 'mock tests',
 'group study',
 'self-paced study',
 'speaking practice',
 'writing practice',
 'writing workshops',
 'vocabulary enhancement',
 'mock interviews',
 'vocabulary building',
 'balanced study routine',
 'online courses',
 'practice',
 'workshops',
 'time management',
 'practice tests',
 'self-study',
 'extensive reading',
 'listening skill improvement',
 'peer feedback',
 'intensive speaking practice',
 'consistent practice',
 'daily practice',
 'intensive course',
 'speaking clubs',
 'online forums',
 'private tutor',
 'structured study plan']

In [356]:
def vectorise_split_columns(data_frame,col_name,initial_values):
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(initial_values)
    cosine_matrix_no_labels = cosine_similarity(tfidf_matrix).round(2)
    cosine_matrix = pd.DataFrame(cosine_similarity(tfidf_matrix).round(2),index=initial_values,columns=initial_values)

    dic_list = []
    
    for i in range(cosine_matrix.shape[0]):
        n = []
        dic = {}
        for j in range(i+1,len(cosine_matrix_no_labels[i])):
            if cosine_matrix_no_labels[i,j] > 0 and cosine_matrix_no_labels[i,j] < 1:
                n.append(j)
        if len(n) > 0:
            dic[initial_values[n[0]]] = initial_values[i]
        else:
            dic[initial_values[i]] = initial_values[i]
        dic_list.append(dic)

    df_modified = data_frame.copy()
    for i in dic_list[::-1]:
        hg = i.copy()
        for j in range(retrieve_no_of_columns_to_split(col_name)):
            b = col_name+" "+str(j)
            df_modified[b] = df_modified[b].map(hg).fillna(df_modified[b])
    return df_modified

In [357]:
df_3 = df_2.copy()
for i in range(len(columns_to_split)):
    df_3 = vectorise_split_columns(df_3,columns_to_split[i],initial_key_preferred[i])

In [358]:
df_3.head()

,Profession,Study Duration (months),Key Strategies 0,Key Strategies 1,Key Strategies 2,Education Level,English Proficiency (Preparation),Practice Hours per Week,Mock Tests Taken,Achieved Desired Score,Preferred Learning Resources 0,Preferred Learning Resources 1,Volunteer Experience,Internship Experience,Recommendations,Networking Efforts,IELTS Score (Overall)
0,Grad Student,3,practice,forums,tutoring,Master's,Advanced,15,5,Yes,online courses,study guides,Yes,Research Assistant,Strong,"LinkedIn, Local Events",8.5
1,IT Professional,4,balanced schedule,time management,speaking with native speakers,Bachelor's,Intermediate,10,3,Yes,online courses,speaking practice,No,IT Consultant,Positive,Online Communities,7.5
2,School Teacher,2,intensive course,online resources,mock tests,Master's,Advanced,20,8,Yes,study guides,language exchange,Yes,Curriculum Developer,Positive,Professional Groups,9.0
3,IT Professional,5,study groups,speaking with native speakers,peer feedback,Bachelor's,Advanced,12,4,Yes,mock exams,speaking clubs,No,Software Developer,Strong,Online Platforms,8.0
4,Eng. Student,6,practice,private tutor,speaking clubs,Bachelor's,Advanced,18,6,Yes,design tutorials,speaking practice,Yes,Research Assistant,Positive,University Events,8.5


In [359]:
cols_dtypes = df_3.dtypes.reset_index()
df_4 = df_3.copy()
string_columns = cols_dtypes["index"][cols_dtypes[0]=="object"].values.tolist()
wwwwww = [string_columns.remove(i) for i in newest_columns]
for ct in string_columns:
    encode_dict = {}
    ctr = 0
    for i in np.sort(df_3[ct].unique()):
        encode_dict[i] = ctr
        ctr += 1
    df_4[ct] = df_4[ct].map(encode_dict)

In [360]:
df_5 = df_4.copy()
for i in columns_to_split:
    num = retrieve_no_of_columns_to_split(i)
    colus = []
    unique_values = []
    for j in range(num):
        column_name = i+" "+str(j)
        colus.append(column_name)
        unique_values += df_4[column_name].unique().tolist()
    unique_values = list(set(unique_values))
    encode_dict = {}
    ctr = 0
    for j in sorted(unique_values):
        encode_dict[j] = ctr
        ctr += 1
    for j in range(num):
        column_name = i+" "+str(j)
        df_5[column_name] = df_5[column_name].map(encode_dict)

In [361]:
df_6 = df_5.copy()
df_6["Practice Hours per Week"] = df_5["Study Duration (months)"] * 4 * df_5["Practice Hours per Week"]
df_6.drop(columns="Study Duration (months)", inplace=True)
df_6.head()

,Profession,Key Strategies 0,Key Strategies 1,Key Strategies 2,Education Level,English Proficiency (Preparation),Practice Hours per Week,Mock Tests Taken,Achieved Desired Score,Preferred Learning Resources 0,Preferred Learning Resources 1,Volunteer Experience,Internship Experience,Recommendations,Networking Efforts,IELTS Score (Overall)
0,6,8,2,14,1,0,180,5,1,9,13,1,10,2,9,8.5
1,9,0,13,11,0,1,160,3,1,9,12,0,7,1,11,7.5
2,13,3,6,5,1,0,160,8,1,13,5,1,2,1,14,9.0
3,9,12,11,7,0,0,240,4,1,7,11,0,11,2,13,8.0
4,3,8,9,10,0,0,432,6,1,1,12,1,10,1,15,8.5


In [362]:
X = df_6.drop(columns=["IELTS Score (Overall)"])
y = df_6["IELTS Score (Overall)"]

mms = MinMaxScaler()
X_res = pd.DataFrame(mms.fit_transform(X),columns=X.columns)
y_res = y.copy()

In [363]:
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.3, random_state=np.random.randint(0,1000))

In [364]:
linear_regr = LinearRegression()
linear_regr.fit(X_train, y_train)
y_pred = np.array(list(map(lambda x: round_to_nearest_ielts_score(x),pd.Series(linear_regr.predict(X_test)))))
y_pred[y_pred>9.0] = 9.0
df_final = pd.DataFrame(columns=["Actual IELTS Score","Predicted IELTS Score"])
df_final["Actual IELTS Score"] = y_test.reset_index(drop=True)
df_final["Predicted IELTS Score"] = y_pred

In [367]:
root_mean_squared_error = round_to_nearest_ielts_score(np.sqrt(mean_squared_error(y_test, y_pred)))
root_mean_squared_error

0.5

In [368]:
df_final

,Actual IELTS Score,Predicted IELTS Score
0,8.0,7.5
1,8.0,7.5
2,7.5,7.5
3,9.0,8.5
4,8.5,8.5
5,8.5,8.5
6,8.0,8.0
7,8.5,8.5
8,8.5,8.5
